In [ ]:
import pandas as pd
import os
PATH = os.getcwd()
print(PATH)
import glob
import datetime
# to check first rows of files
from csv import reader
# display info options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image


In [ ]:
def read_mch_info(file):
    i = 0
    with open(file, 'r') as csv_file:
        csv_reader = reader(csv_file, delimiter=';')
        mch_info = []
        for row in csv_reader:
            # row variable is a list that represents a row in csv
            mch_info.append(row[0])
            i += 1
            if i == 3:
                break

    return(mch_info)

In [ ]:
def concat_files(AllFilesNames):
   # Create an empty list to store the dataframes
    ListDataframe = list()

    # Total file number
    NoFiles = len(AllFilesNames)

    # For each file in the specified directory import the data and add it in the list
    for Filename in AllFilesNames:
        ListDataframe.append(pd.read_csv(Filename, sep=';', skiprows=3, decimal=',', encoding='unicode_escape'))

    # Concatenate the files in the list in unique dataframe
    DF_Data = pd.concat(ListDataframe, axis=0, ignore_index=True)

    # DateTime
    DF_Data['DateTime'] = pd.to_datetime(DF_Data['DateTime'], format="%Y-%m-%d %H:%M:%S")
    # ordering ascending
    DF_Data = DF_Data.sort_values(by='DateTime', ascending=True).reset_index(drop=True)

    #Remove duplicates
    DF_Data.drop_duplicates(keep=False, inplace=True)

    return(DF_Data)

In [ ]:
# Folder name / TODO: GET FROM USER
foldername = "FCM_B_C/FCM_C/LogExport202107061025/"

In [ ]:
# Define files path
Name = "S?*.csv"
# Read all the files name in the directory
DataFiles = glob.glob(foldername + Name)

# Define files path
Name = "A?*.csv"
# Read all the files name in the directory
AlarmFiles = glob.glob(foldername + Name)

# Define files path
Name = "E?*.csv"
# Read all the files name in the directory
EventFiles = glob.glob(foldername + Name)

print(DataFiles)
print(AlarmFiles)
print(EventFiles)

In [ ]:
def check_files (files):
    # Check Machine information
    mch_info_check = read_mch_info(files[0])

    for file in (files):
        mch_info = read_mch_info(file)
        if mch_info != mch_info_check:
            print('File ' + file + ' does not correspond to the same machine')
            print(mch_info_check)
            return 0, mch_info_check
    
    print('All files correspond to the same machine')
    print(mch_info_check)
    return 1, mch_info_check

In [ ]:
flag_files, mch_info = check_files(DataFiles + AlarmFiles + EventFiles)
if flag_files:
    print('all good')

In [ ]:
LogsStandard = concat_files(DataFiles)
print(LogsStandard.columns)
print(LogsStandard.shape)

LogsAlarms = concat_files(AlarmFiles)
print(LogsAlarms.columns)
print(LogsAlarms.shape)

LogsEvents = concat_files(EventFiles)
print(LogsEvents.columns)
print(LogsEvents.shape)


In [ ]:
# Format LogsStandard
LogsStandard = LogsStandard[['DateTime','GpsPos', 'ChangeOverInProgress','F60InAutoMode','CV1_Position', 'CV2_Position',
                  'CV3_Position', 'CV4_Position', 'CV5_Position', 'SupplyCurrentPump',
       'CircCurrentPump', 'CurrentControl', 'CurrentFilter',
       'PT1', 'PT2', 'TT1', 'TT2', 'TargetTemperature',
       'TemperatureLowLimit', 'TemperatureHighLimit', 'VT', 'TargetViscosity',
       'ViscosityLowLimit', 'ViscosityHighLimit', 'Density', 'FM1_MassFlow',
       'FM1_Density', 'FM1_Temperature', 'FM2_MassFlow', 'FM2_Density',
       'FM2_Temperature', 'FM3_MassFlow', 'FM3_Density', 'FM3_Temperature',
       'FM4_MassFlow', 'FM4_Density', 'FM4_Temperature', 'FT_VolumeFlow',
       'FT_MassFlow', 'FT_Density', 'FT_Temperature', 'SO2', 'CO2', 'SC','DPT_AI']]

# Columns as float
LogsStandard.iloc[:,13:] = LogsStandard.iloc[:,13:].astype(float)

LogsStandard[['CV1_Label','CV2_Label','CV3_Label','CV4_Label','CV5_Label']] = LogsStandard[['CV1_Position','CV2_Position','CV3_Position','CV4_Position','CV5_Position']]

LogsStandard[['CV1_Label']] = LogsStandard[['CV1_Label']] .replace({ 
    0: "Both LS activated",
    1: "Fuel 1 Position",
    2: "No LS activated",
    3: "Other fuel Position"
})

LogsStandard[['CV2_Label']] = LogsStandard[['CV2_Label']] .replace({ 
    0: "Both LS activated",
    1: "Fuel 2 Position",
    2: "No LS activated",
    3: "Other Fuel Position"
})

LogsStandard[['CV3_Label']] = LogsStandard[['CV3_Label']] .replace({ 
    0: "Both LS activated",
    1: "Fuel 3 Position",
    2: "No LS activated",
    3: "Fuel 4 Position"
})

LogsStandard[['CV4_Label']] = LogsStandard[['CV4_Label']] .replace({ 
    0: "Both LS activated",
    1: "Heater Position",
    2: "No LS activated",
    3: "Cooler Position"
})

LogsStandard[['CV5_Label']] = LogsStandard[['CV5_Label']] .replace({ 
    0: "Both LS activated",
    1: "Cooler Position",
    2: "No LS activated",
    3: "Bypass Position"
})

LogsStandard.dtypes

In [ ]:
LogsAlarms = LogsAlarms[['DateTime', 'AlarmNumber']]

LogsAlarms['Label'] = LogsAlarms['AlarmNumber'] 

LogsAlarms[['Label']] = LogsAlarms[['Label']] .replace({ 
    0: "PLC battery low / Not present",
    1: "Dynamic I/O Configuration Error",
    2: "Dynamic I/O Configuration In Progress",
    3: "Dynamic I/O Configuration Done",
    4: "Invalid GPS Sig-nal ",
    5: "I/O Module Error",
    102: "Emergency stop",
    103: "Power failure",
    104: "Instrument Mod-bus Communi-cation Error",
    105: "UPS Battery Low",
    106: "Local HMI com-munication lost",
    107: "Remote HMI communication lost",
    200: "Changeover Valve 1 Alarm",
    201: "Changeover Valve 2 Alarm",
    202: "Changeover Valve 3 Alarm",
    203: "Changeover Finished",
    300: "SCT sensor fault, signal missing",
    301: "Fuel consump-tion too low for blending",
    400: "Supply Pump 1 failure",
    401: "Supply Pump 2 failure",
    402: "Supply pump 1 Not Available",
    403: "Supply pump 2 Not Available",
    404: "PT1 limit high",
    405: "PT1 limit low",
    406: "PT1 sensor fault, signal missing",
    408: "Supply Pump switch over done",
    409: "Automatic switch of Supply Pumps in a short time",
    410: "Standby Supply Pump run time reset to zero",
    411: "Supply Pump switch time elapsed",
    412: "Supply Pump switching failed",
    413: "Standby Supply Pump not avail-able Please re-store it",
    414: "Warning/Alarm on VFD 1",
    415: "Warning/Alarm on VFD 2",
    500: "Circulation Pump 1 failure",
    501: "Circulation Pump 2 failure",
    502: "Circulation pump 1 Not Available",
    503: "Circulation pump 2 Not Available",
    504: "PT2 limit high",
    505: "PT2 limit low",
    506: "PT2 sensor fault, signal missing",
    508: "Circulation Pump switch-over done",
    509: "Automatic switch of Circulation Pumps in a short time",
    510: "Standby Circulation Pump run time reset to zero",
    511: "Circulation Pump switch time elapsed",
    512: "Circulation Pump switching failed ",
    513: "Standby Circula-tion Pump not available. Please restore it.",
    600: "Automatic Filter Differential Pressure high",
    601: "Automatic Filter Failure",
    602: "DPT Filter Fault",
    800: "Mixing tank low level",
    900: "Electric Heater 1-Fault",
    901: "Electric Heater 2-Fault",
    902: "Fuel Temperature High",
    903: "Fuel Temperature Low",
    905: "Chiller Fault",
    906: "Viscosity not reached during Changeover.",
    907: "TT1 sensor fault, signal missing",
    908: "TT2 sensor fault, signal missing",
    911: "CV4 Alarm",
    912: "CV5 Alarm",
    913: "Switched to Vis-cosity control",
    914: "Switched to Temperature control",
    915: "Temperature control out of order, will be switched off within 5 minutes",
    916: "Temperature control switched off",
    917: "Very high temperature reached",
    1000: "Viscosity Sensor Fault",
    1002: "Density Sensor Fault",
    1004: "Fuel viscosity High limit alarm",
    1005: "Fuel viscosity Low limit alarm"
})

LogsAlarms.dtypes

In [ ]:
LogsEvents = LogsEvents[['DateTime', 'GpsPos', 'EventNumber', 'Data']]
LogsEvents['Label'] = LogsEvents['EventNumber'] 

LogsEvents[['Label']] = LogsEvents[['Label']] .replace({0 : 'FCM Started',
                                                        1 : 'FCM Stopped',
                                                        2 : 'Changeover initiated',
                                                        3 : 'Changeover Finsihed',
                                                        4 : 'Any device in Manual Mode',
                                                        5 : 'All device in Auto Mode',
                                                        6 : 'P401 SPump value changed',
                                                        7 : 'P501 CPump value changed',
                                                        8 : 'P903 Heater In Use value changed',
                                                        30 : 'Auto Mode Selected',
                                                        31 : 'Manual Mode Selected'})

LogsEvents.loc[(LogsEvents['EventNumber'] == 6) |
               (LogsEvents['EventNumber'] == 7) | 
               (LogsEvents['EventNumber'] == 8), 'Label'] = LogsEvents['Label'].astype(str) + ": " + LogsEvents['Data'].astype(str)


LogsEvents.head()

# Change Overs

In [ ]:
# Identify when Change Over Started and Finished
# ChangeOverInProgress = O : no change
# ChangeOverInProgress = 1 : started
# ChangeOverInProgress = -1 : finished

# Create column with value change
LogsStandard['ChangeoverCMDchange'] = LogsStandard['ChangeOverInProgress'].diff()

# memorize starting DateTimes
COstart = LogsStandard[LogsStandard['ChangeoverCMDchange'] == 1]['DateTime'].tolist()

# memorize finishing DateTimes
COfinish = LogsStandard[(LogsStandard['ChangeoverCMDchange'] == -1)]['DateTime'].tolist()

COs = []
for i in range(len(COfinish)):
    duration = COfinish[i]-COstart[i]
    if duration > datetime.timedelta(minutes = 1):
        COs.append({'Start': COstart[i], 'Finish': COfinish[i], 'Duration': duration})

print('In the logs imported there are ' + str(len(COs)) + ' changeovers')
for CO in COs:
    print('- From ' + str(CO['Start']) + ' to ' + str(CO['Finish']) + '. Duration: ' + str(CO['Duration'])) 


In [ ]:
def ChangeOverToDF(CO):
    delta = datetime.timedelta(minutes = 20)
    df = LogsStandard[(LogsStandard['DateTime'] >= CO['Start']-delta) & (LogsStandard['DateTime'] <= CO['Finish']+delta)]
    return(df)

# PLOTS

In [ ]:
greens = ['rgba(0, 128, 0, 0.1)', #green opacity 10%
          'rgba(196, 180, 84, 1)', #vegas gold
          'rgba(64, 224, 208, 1)', #turquoise
          'rgba(69, 75, 27, 1)', #army green
          'rgba(147, 197, 114, 1)', #pistachio
          'rgba(8, 143, 143, 1)', #citrine
        ]

blues = ['rgba(0, 71, 171, 1)', #cobalt blue
        'rgba(70, 130, 180, 1)' #steel blue
        ]

reds = ['rgba(255, 87, 51, 1)', #cherry
        'rgba(250, 160, 160, 1)', #pastel red
        ]

purples = ['rgba(191, 64, 191, 1)', #bright purple
           'rgba(93, 63, 211, 1)', #iris
           'rgba(127, 0, 255, 1)', #violet
           'rgba(149, 53, 83, 1)', #red purple
           'rgba(218, 112, 214, 1)' #mulberry
           ]

comp1 = ['rgba(29, 157, 228, 1)', #blue*
           'rgba(228, 100, 29, 1)', #orange*
           'rgba(63, 166, 121, 1)', #green*
           ]

ALcolors = ['rgba(17, 56, 127, 1)', #AL blue
            'rgba(0, 0, 0, 1)', #AL white
            'rgba(220, 146, 118, 1)', #AL earth
            'rgba(254, 205, 96, 1)', #AL sun
            'rgba(147, 199, 198, 1)', #AL water
            'rgba(0, 127, 200, 1)', #AL innovation
            ]

In [ ]:
def Plot_ChangeOver_simple(df):
    fig = go.Figure()

    # --------------------------------------------------- Valves
    greens = ['rgba(0, 128, 0, 0.1)', #green opacity 10%
            'rgba(196, 180, 84, 1)', #vegas gold
            'rgba(64, 224, 208, 1)', #turquoise
            'rgba(69, 75, 27, 1)', #army green
            'rgba(147, 197, 114, 1)', #pistachio
            'rgba(8, 143, 143, 1)', #citrine
            ]

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ChangeOverInProgress'].astype(bool),
        name="Change Over InProgress",
        fill='tozeroy', mode='none', fillcolor = greens[0],    
        yaxis="y4"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        line=dict(color = greens[1]), 
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV2_Position'],
        name="CV2 Position",
        line=dict(color = greens[2]),
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV3_Position'],
        name="CV3 Position",
        line=dict(color = greens[3]),    
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV4_Position'],
        name="CV4 Position",
        line=dict(color = greens[4]), 
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV5_Position'],
        name="CV5 Position",
        line=dict(color = greens[5]),
        legendgroup="Valves",legendgrouptitle_text="Valves",
        visible='legendonly',
        yaxis="y3"
    ))

    # ---------------------------------------------------Temperature
    reds = ['rgba(255, 87, 51, 1)', #cherry
            'rgba(250, 160, 160, 1)' #pastel red
            ]
    if (df['TT1'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT1'],
            name='TT1 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ))

    if (df['TT2'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT2'],
            name='TT2 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature',
        line=dict(color = reds[1]),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureLowLimit'],
        name='Temp Low Limit',
        line=dict(color = reds[1], dash='dot'),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureHighLimit'],
        name='Temperature High Limit',
        line=dict(color = reds[1], dash='dot'),
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ))

    # --------------------------------------------------- Viscosity
    blues = ['rgba(0, 71, 171, 1)', #cobalt blue
            'rgba(70, 130, 180, 1)' #steel blue
            ]

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Viscosity',
        line=dict(color = blues[0]),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetViscosity'],
        name='Target Viscosity',
        line=dict(color = blues[1]),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityLowLimit'],
        name='Viscosity Low Limit',
        line=dict(color = blues[1], dash='dot'),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityHighLimit'],
        name='Viscosity High Limit',
        line=dict(color = blues[1], dash='dot'),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity",
        yaxis="y2"))

    # Create axis objects
    fig.update_layout(
        xaxis=dict(domain=[0.1, 0.9]), #compress x axis 10% left an right

        yaxis=dict(
            title="Temperature"),
        yaxis2=dict(
            title="Viscosity",
            anchor="free",
            overlaying="y",
            side="left",
            position=0
        ),
        yaxis3=dict(
            title="CV Position",
            anchor="x",
            overlaying="y",
            side="right"
        ),
        yaxis4=dict(
            title="Change Over",
            anchor="free",
            overlaying="y",
            side="right",
            position=1
        ),
        legend=dict(orientation="v", x = 1.1)
    )

    # Update layout properties
    fig.update_layout(
        title_text=("ChangeOver and CVs"),
        legend=dict(groupclick="toggleitem") #avoid grouping all traces
    )

    return (fig)

In [ ]:
def Plot_ChangeOver(df, mch_info):
    mindate = df['DateTime'].min()
    maxdate = df['DateTime'].max()
    print(mindate,maxdate)

    alm = pd.DataFrame()
    alm = LogsAlarms[(LogsAlarms['DateTime'] > mindate) & (LogsAlarms['DateTime'] <= maxdate)]

    eve = pd.DataFrame()
    eve = LogsEvents[(LogsEvents['DateTime'] > mindate) & (LogsEvents['DateTime'] <= maxdate)]

    fig = make_subplots(
        rows=8, cols=1,
        specs=[[{"rowspan": 4, "colspan": 1, "secondary_y": True}], # 4 Rows for Visc and Temp Trends
            [None],
            [None],
            [None],
            [{"secondary_y": True}],  # Valves position & ChangeOverON
            [{"secondary_y": True}],  # FT & Density
            [{"secondary_y": True}],  # PT & Alarms
            [{"secondary_y": True}]], # Events & Alarms
            shared_xaxes=True,
        print_grid=True)

    # --------------------------------------------------------------------------------------------- R1-4 C1
    plot_row= 1
    plot_col= 1

    fig.update_yaxes(title_text="Temperature", 
                    title_font_color= reds[0],
                    color= reds[0],row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Viscosity", 
                    title_font_color= blues[0],
                    color= blues[0], row=plot_row, secondary_y=True)

    # ---------------------------------------------------Temperature
    if (df['TT1'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT1'],
            name='TT1 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ), row=plot_row, col=plot_col, secondary_y=False)

    if (df['TT2'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['TT2'],
            name='TT2 - Act Temp',
            line=dict(color = reds[0]),
            legendgroup="Temperature",legendgrouptitle_text="Temperature",
        ), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetTemperature'],
        name='Target Temperature',
        line=dict(color = reds[1]),line_shape='hv',
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureLowLimit'],
        name='Temp Low Limit',
        line=dict(color = reds[1], dash='dot'),line_shape='hv',
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TemperatureHighLimit'],
        name='Temperature High Limit',
        line=dict(color = reds[1], dash='dot'),line_shape='hv',
        legendgroup="Temperature",legendgrouptitle_text="Temperature",
    ), row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------- Viscosity

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['VT'],
        name='Viscosity',
        line=dict(color = blues[0]),
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['TargetViscosity'],
        name='Target Viscosity',
        line=dict(color = blues[1]),line_shape='hv',
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityLowLimit'],
        name='Viscosity Low Limit',
        line=dict(color = blues[1], dash='dot'),line_shape='hv',
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ViscosityHighLimit'],
        name='Viscosity High Limit',
        line=dict(color = blues[1], dash='dot'),line_shape='hv',
        legendgroup="Viscosity",legendgrouptitle_text="Viscosity"),
        row=plot_row, col=plot_col, secondary_y=True)

    # --------------------------------------------------------------------------------------------- R5 C1
    plot_row= 5
    plot_col= 1

    fig.update_yaxes(visible = False,
                    title_text="True/False",
                    row=plot_row, secondary_y=True)
    fig.update_yaxes(title_text="Valve Position",
                    row=plot_row, secondary_y=False)
    # --------------------------------------------------- Bools
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['ChangeOverInProgress'].astype(bool),
        name="Change Over InProgress",
        fill='tozeroy', mode='none', fillcolor = greens[0],    
        line_shape='hv'
    ), row=plot_row, col=plot_col, secondary_y=True)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV1_Position'],
        name="CV1 Position",
        line=dict(color = ALcolors[0]), 
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV1_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV2_Position'],
        name="CV2 Position",
        line=dict(color = ALcolors[2]),
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV2_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV3_Position'],
        name="CV3 Position",
        line=dict(color = ALcolors[3]),    
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV3_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV4_Position'],
        name="CV4 Position",
        line=dict(color = ALcolors[4]), 
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV4_Label']), row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['CV5_Position'],
        name="CV5 Position",
        line=dict(color = ALcolors[5]),
        legendgroup="Valves",legendgrouptitle_text="Valves", line_shape='hv',
        hovertext=df['CV5_Label']), row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------------------------------------------------- R6 C1
    plot_row= 6
    plot_col= 1

    fig.update_yaxes(title_text="Flow",
                    row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Density",
                    row=plot_row, secondary_y=True)
    # --------------------------------------------------- Flow Meter
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['FT_MassFlow'],
        name='FT_MassFlow',
        line=dict(color = purples[0]),
        legendgroup="FT",legendgrouptitle_text="Flow"),
        row=plot_row, col=plot_col, secondary_y=False)
        
    if (df['FM1_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM1_MassFlow'],
            name='FM1 MassFlow',
            line=dict(color = purples[1]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)

    if (df['FM2_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM2_MassFlow'],
            name='FM2 MassFlow',
            line=dict(color = purples[2]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)
        
    if (df['FM3_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM3_MassFlow'],
            name='FM3 MassFlow',
            line=dict(color = purples[3]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)

    if (df['FM4_MassFlow'] != 0).all():
        fig.add_trace(go.Scatter(
            x=df['DateTime'],
            y=df['FM4_MassFlow'],
            name='FM4 MassFlow',
            line=dict(color = purples[4]),
            legendgroup="FT",legendgrouptitle_text="Flow"),
            row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------- Density
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['Density'],
        name='Density',
        line=dict(color = comp1[2]),
        legendgroup="Density",legendgrouptitle_text="Density"),
        row=plot_row, col=plot_col, secondary_y=True)
        
    # --------------------------------------------------------------------------------------------- R7 C1
    plot_row= 7
    plot_col= 1

    fig.update_yaxes(title_text="Pressure",
                    row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Alarms",
                    row=plot_row, secondary_y=True)
    # --------------------------------------------------- Pressures
    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['PT1'],
        name='PT1',
        line=dict(color = comp1[0]),
        legendgroup="PT",legendgrouptitle_text="Pressure"),
        row=plot_row, col=plot_col, secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df['DateTime'],
        y=df['PT2'],
        name='PT2',
        line=dict(color = comp1[1]),
        legendgroup="PT",legendgrouptitle_text="Pressure"),
        row=plot_row, col=plot_col, secondary_y=False)

    # --------------------------------------------------- Alarms
    p_alm = alm[(alm['AlarmNumber'] > 399) & (alm['AlarmNumber'] <= 499)] # Pumps Alarms A4xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Pressure Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="SPump Alarms"),
                        row=plot_row, col=plot_col,secondary_y=True)   
        
    p_alm = alm[(alm['AlarmNumber'] > 499) & (alm['AlarmNumber'] <= 599)] # Pumps Alarms A5xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Pressure Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="CPump Alarms"),
                        row=plot_row, col=plot_col,secondary_y=True)    

    # --------------------------------------------------------------------------------------------- R8 C1
    plot_row= 8
    plot_col= 1

    fig.update_yaxes(title_text="Events",
                    row=plot_row, secondary_y=False)
    fig.update_yaxes(title_text="Alarms",
                    row=plot_row, secondary_y=True)
    # --------------------------------------------------- Events
    if not eve.empty:
        fig.add_trace(go.Scatter(x=eve['DateTime'], y=eve['EventNumber'].astype(int),
                                name='Event Number',
                                mode='markers',marker_symbol='star',
                                marker_line_color=ALcolors[5], marker_color=ALcolors[5],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Events",legendgrouptitle_text="Events",
                                hovertext=eve['Label']),
                        row=plot_row, col=plot_col,secondary_y=False)
        

    # --------------------------------------------------- Alarms
    p_alm = alm[(alm['AlarmNumber'] < 199)] # System Alarms A0xx to A1xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='System Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True)     
        

    p_alm = alm[(alm['AlarmNumber'] > 199) & (alm['AlarmNumber'] <= 299)] # ChangeOver Alarms A2xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='ChangeOver Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True)     
        
    p_alm = alm[(alm['AlarmNumber'] > 299) & (alm['AlarmNumber'] <= 399)] # Blending Alarms A3xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Blending Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 599) & (alm['AlarmNumber'] <= 699)] # Filter Alarms A6xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Filter Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 799) & (alm['AlarmNumber'] <= 899)] # Mixing Tank Alarms A8xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='Mixing Tank',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 899) & (alm['AlarmNumber'] <= 999)] # TempControl Alarms A9xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='TempControl Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 
        
    p_alm = alm[(alm['AlarmNumber'] > 999) & (alm['AlarmNumber'] <= 1099)] # ViscMeas Alarms A9xx
    if not p_alm.empty:
        fig.add_trace(go.Scatter(x=p_alm['DateTime'], y=p_alm['AlarmNumber'].astype(str),
                                name='ViscMeas Alarms',
                                mode='markers', marker_symbol='x',
                                marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                                marker_line_width=1, marker_size=8,
                                legendgroup="Alarms",legendgrouptitle_text="Alarms",
                                hovertext=p_alm['Label']),
                        row=plot_row, col=plot_col,secondary_y=True) 

    # Update layout properties
    fig.update_layout(hovermode="x unified", hoverlabel=dict(bgcolor='rgba(255,255,255,0.75)', namelength = -1, font=dict(color='black')),  
        legend=dict(groupclick="toggleitem"), #avoid grouping all traces #orientation="v", x = 1.1, 
        title_text=(mch_info[0] + " | " + mch_info[1] + " | " + mch_info[2]) , title_x=0.5
    )


    # Add image
    from PIL import Image
    alLogo = Image.open(PATH + "\VisuaLite\\resources\ALlogo.png")
    fig.add_layout_image(
        dict(
            source=alLogo,
            xref="paper", yref="paper",
            x=0, y=1.025,
            sizex=0.14, sizey=0.14,
            xanchor="left", yanchor="bottom"
        )
    )

    return(fig)


In [ ]:
df = ChangeOverToDF(COs[1])

fig = Plot_ChangeOver(df, mch_info)
#fig.write_html("ChangeOver_test4ROWS.html", config={'displaylogo': False})
#fig.update_layout(autosize=False, width=1000, height=800)
fig.show(config={'displaylogo': False})

fig2 = Plot_ChangeOver_simple(df)
#fig2.write_html("ChangeOver_test1ROW.html", config={'displaylogo': False})
#fig2.update_layout(autosize=False, width=1000, height=800)
fig2.show(config={'displaylogo': False})


# STAMPA REPORT

Tutti Par ChangeOver, Fuel, Temp, Visc

In [ ]:
# ***If selected tab is available, add NEXT button to go to print results

# Try to create a PDF starting from a word with Alfa Laval brand, and let's add some plot images, text and print it as pdf
    # how to save plot images
    # how it should look for the user, how to select the report type
    # how to modify from python a word file (format it and insert imgs)

# Search for an alarm or event

In [ ]:
# Simulate tkinter interactions with jupyter widgets

selected_alm = None
selected_eve = None
selected_type = None

from ipywidgets import widgets

# ---------------------------------------------------------------------------------------------------------- Functions widgets callbacks
# Function to display the selected value for dropdown1
def on_dropdown1_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global selected_alm 
        selected_alm = change['new']
        print(f"Selected from dropdown1: {change['new']}")

# Function to display the selected value for dropdown2
def on_dropdown2_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global selected_eve 
        selected_eve = change['new']
        print(f"Selected from dropdown2: {change['new']}")

# button function
def on_button_click(b):
    if selected_type == 'Alarm':
        x = selected_alm
    elif selected_type == 'Event':
        x = selected_eve

    print("Let's plot " + x)

# Function to display the selected value
def on_radio_button_change(change): 
    global selected_type
    selected_type = change['new']
    if selected_type == 'Alarm':
        dropdown1.layout.visibility = 'visible'
        dropdown2.layout.visibility = 'hidden'
    elif selected_type == 'Event':
        dropdown1.layout.visibility = 'hidden'
        dropdown2.layout.visibility = 'visible'
    print(f"Selected: {selected_type}")

# ---------------------------------------------------------------------------------------------------------- Options created
# List of options for the radio buttons
options = ['Event', 'Alarm']

# Create a list of options for the dropdown
LogsAlarms['Alm_Code_Label'] = "A" + LogsAlarms['AlarmNumber'].astype(str) + "_" + LogsAlarms['Label'] 
options1= LogsAlarms['Alm_Code_Label'].unique().tolist()
options1.sort()

LogsEvents['Evn_Code_Label'] = "E" + LogsEvents['EventNumber'].astype(str) + "_" + LogsEvents['Label'] 
options2 = LogsEvents['Evn_Code_Label'].unique().tolist()
options2.sort()

# ---------------------------------------------------------------------------------------------------------- Create widgets

# Create the dropdown widgets
dropdown1 = widgets.Dropdown(options=options1, description='Select an option:')
dropdown2 = widgets.Dropdown(options=options2, description='Select an option:')

# Create a radio buttons widget
radio_buttons = widgets.RadioButtons(options=options, description='Select an option:')

# Create a button widget
button = widgets.Button(description="Click me")

# ---------------------------------------------------------------------------------------------------------- Link widgets with functions
# Link the function to the radio buttons' value
radio_buttons.observe(on_radio_button_change, names='value')

# Link the functions to the dropdowns' values
dropdown1.observe(on_dropdown1_change, names='value')
dropdown2.observe(on_dropdown2_change, names='value')

# Link the function to the button value
button.on_click(on_button_click)

# ---------------------------------------------------------------------------------------------------------- display widgets
display(radio_buttons)
display(dropdown1)
display(dropdown2)
display(button)

In [ ]:
text = "A400_Supply Pump 1 failure"
alm_number= int(text[1:text.index('_')]) #text.index('_') returns position of _ char / or int(text.split('_')[0][1:]) this function splits string by _ 

# memorize Selected alarm/event DateTimes
timestamps = LogsAlarms[LogsAlarms['AlarmNumber'] == alm_number]['DateTime'].tolist()
for t in timestamps:
    text=str(t.date()) + " at " + str(t.time())
    print(text)
# dropdown con intervalo di tempo prima e dopo l'evento selezionato


# Custom plots function

In [ ]:
units = {
    'DateTime' : "datetime",
    'GpsPos': "gps",
    'ChangeOverInProgress' : "bool",
    'F60InAutoMode' : "bool", 
    'CV1_Position' : "valve_pos",
    'CV2_Position' : "valve_pos",
    'CV3_Position' : "valve_pos",
    'CV4_Position' : "valve_pos",
    'CV5_Position' : "valve_pos",
    'SupplyCurrentPump' : "int",
    'CircCurrentPump' : "int",
    'CurrentControl' : "int",
    'CurrentFilter' : "int",
    'PT1' : "bar",
    'PT2' : "bar",
    'TT1' : "degreesC",
    'TT2' : "degreesC",
    'TargetTemperature' : "degreesC",
    'TemperatureLowLimit' : "degreesC",
    'TemperatureHighLimit' : "degreesC",
    'VT': "cSt",
    'TargetViscosity': "cSt",
    'ViscosityLowLimit': "cSt",
    'ViscosityHighLimit': "cSt",
    'Density': "density",
    'FM1_MassFlow': "mass_flow",
    'FM1_Density': "density",
    'FM1_Temperature' : "degreesC",
    'FM2_MassFlow': "mass_flow",
    'FM2_Density' : "density",
    'FM2_Temperature' : "degreesC",
    'FM3_MassFlow': "mass_flow",
    'FM3_Density' : "density",
    'FM3_Temperature' : "degreesC",
    'FM4_MassFlow': "mass_flow",
    'FM4_Density' : "density",
    'FM4_Temperature' : "degreesC",
    'FT_VolumeFlow': "vol_flow",
    'FT_MassFlow': "mass_flow",
    'FT_Density' : "density",
    'FT_Temperature' : "degreesC",
    'SO2': "float",
    'CO2': "float",
    'SC': "float",
    'DPT_AI': "float",
    'AlarmNumber': "alarms",
    'EventNumber': "events",
}
print(units)
import json
with open('Visualite/resources/units.txt', 'r') as file:
    units = json.load(file)

print(units)


In [ ]:
text = "A400_Supply Pump 1 failure"
alm_number= int(text[1:text.index('_')]) #text.index('_') returns position of _ char / or int(text.split('_')[0][1:]) this function splits string by _ 

# memorize Selected alarm/event DateTimes
timestamps = LogsAlarms[LogsAlarms['AlarmNumber'] == alm_number]['DateTime'].tolist()
for t in timestamps:
    text=str(t.date()) + " at " + str(t.time())
    print(text)
    print(t)
# dropdown con intervalo di tempo prima e dopo l'evento selezionato


In [ ]:
def classify_cols(selected):
    filter_unit_cols = {col : unit for col, unit in units.items() if col in selected}
    # {key_expression: value_expression for item in iterable if condition}

    classified_cols = {}
    for col, unit in filter_unit_cols.items():
        if unit not in classified_cols:
            classified_cols[unit] = [] #if first column of this unit, create key and an empty array as value
        classified_cols[unit].append(col)

    return classified_cols

In [ ]:
def date_limits (timestamp, lower, upper):
    date1 = timestamp
    date2 = timestamp

    if lower == "1 hour":
        date1 = timestamp - datetime.timedelta(hours=1)
    elif lower == "2 hours":
        date1 = timestamp - datetime.timedelta(hours=2)
    elif lower == "4 hours":
        date1 = timestamp - datetime.timedelta(hours=4)
    elif lower == "8 hours":
        date1 = timestamp - datetime.timedelta(hours=8)
    elif lower == "1 day":
        date1 = timestamp - datetime.timedelta(days=1)

    if upper == "1 hour":
        date2 = timestamp + datetime.timedelta(hours=1)
    elif upper == "2 hours":
        date2 = timestamp + datetime.timedelta(hours=2)
    elif upper == "4 hours":
        date2 = timestamp + datetime.timedelta(hours=4)
    elif upper == "8 hours":
        date2 = timestamp + datetime.timedelta(hours=8)
    elif upper == "1 day":
        date2 = timestamp + datetime.timedelta(days=1)
    
    return date1, date2

In [ ]:
# Get the path to the current script
script_path = "c:\_Projects\_REPO\_otherREPO\DataAnalysis\VisuaLite\modules"

def custom_plot1 (dfs, dfa, dfe, cols, date1, date2, tittle): # 1 plot with multiple axis

    dfs = dfs[(dfs['DateTime'] >= date1) & (dfs['DateTime'] <= date2)]
    dfa = dfa[(dfs['DateTime'] >= date1) & (dfa['DateTime'] <= date2)]
    dfe = dfe[(dfs['DateTime'] >= date1) & (dfe['DateTime'] <= date2)]

    cols2 = classify_cols(cols)

    fig = make_subplots(rows=len(cols2), cols=1, shared_xaxes=True, vertical_spacing=0.02)

    for i, (unit, cols) in enumerate(cols2.items()):
        print(i, unit)
        fig.update_yaxes(title_text=unit,row=i+1)

        for col in cols:
            if col == 'AlarmNumber':
                trace = go.Scatter(x=dfa['DateTime'], y=dfa['AlarmNumber'].astype(int),
                        name=col,
                        mode='markers', marker_symbol='x',
                        marker_line_color=ALcolors[2], marker_color=ALcolors[3],
                        marker_line_width=1, marker_size=8,
                        legendgroup=unit,legendgrouptitle_text=unit,
                        hovertext=dfa['Label'])
                 
            elif col == 'EventNumber':
                trace = go.Scatter(x=dfe['DateTime'], y=dfe['EventNumber'].astype(int),
                        name='Event Number',
                        mode='markers',marker_symbol='star',
                        marker_line_color=ALcolors[5], marker_color=ALcolors[5],
                        marker_line_width=1, marker_size=8,
                        legendgroup="Events",legendgrouptitle_text="Events",
                        hovertext=dfe['Label'])
            else:
                trace = go.Scatter(
                    x=dfs['DateTime'],
                    y=dfs[col],
                    name=col,
                    legendgroup=unit,legendgrouptitle_text=unit,
                )
                
            fig.add_trace(trace, row=i+1, col=1)

    # Update layout properties
    fig.update_layout(hovermode="x unified", hoverlabel=dict(bgcolor='rgba(255,255,255,0.75)', namelength = -1, font=dict(color='black')),  
        legend=dict(groupclick="toggleitem"), #avoid grouping all traces #orientation="v", x = 1.1, 
        title_text=tittle , title_x=0.5
    )

    # Add image
    img_path = os.path.join(script_path, '..', 'resources', 'ALlogo.png')
    alLogo = Image.open(img_path)
    fig.add_layout_image(
        dict(
            source=alLogo,
            xref="paper", yref="paper",
            x=0, y=1.025,
            sizex=0.14, sizey=0.14,
            xanchor="left", yanchor="bottom"
        )
    )
    
    return fig

In [ ]:
selected = ['PT1', 'PT2', 'VT', 'TargetViscosity', 'Density', 'EventNumber', 'AlarmNumber']
time = datetime.datetime.strptime('2021-06-30 18:18:08', '%Y-%m-%d %H:%M:%S')
date1, date2 = date_limits(time,"1 day", "2 day")
fig = custom_plot1 (LogsStandard, LogsAlarms, LogsEvents, selected, date1, date2, "CustomTittle")
fig.show()

In [ ]:
x =LogsStandard.columns.tolist()
print(x)

In [ ]:
now_dt = datetime.datetime.now()
format_dt = now_dt.strftime('%Y.%m.%d_%H%M%S')
print(format_dt)

# Matplotlib plot to show in third tab

In [ ]:
import matplotlib.pyplot as plt

# Define your custom style
custom_dark_style = {
    'axes.facecolor': '#24242424',  # Custom background color / DEFAULT '#1e1e1e'
    'axes.edgecolor': '#ffffff',  # Color of axes edges / DEFAULT '#ffffff'
    'axes.labelcolor': '#ffffff',  # Color of axes labels / DEFAULT 
    'text.color': '#ffffff',  # Color of text / DEFAULT '#ffffff'
    'xtick.color': '#ffffff',  # Color of x-axis ticks / DEFAULT '#ffffff'
    'ytick.color': '#ffffff',  # Color of y-axis ticks / DEFAULT '#ffffff'
    'figure.facecolor': '#1e1e1e',  # Figure background color / DEFAULT '#1e1e1e'
    'figure.edgecolor': '#1e1e1e',  # Figure edge color / DEFAULT '#1e1e1e'
    'axes.grid': True,  # Show grid / DEFAULT True
    'grid.color': '#333333',  # Color of grid lines / DEFAULT '#333333'
}
plt.style.use(custom_dark_style)
#plt.style.use('default')
#plt.style.use('dark_background')

# Create a two-row plot
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 6), gridspec_kw={'height_ratios': [1, 5]})

# ---------------------------------------------------------------------------- Row 1: Line plots
axes[0].set_title('Date range of Log files')
axes[0].plot(LogsStandard['DateTime'], [3] * len(LogsStandard), color='#FFCC00', linewidth=8, label='LogsStandard')

axes[0].plot(LogsAlarms['DateTime'], [2] * len(LogsAlarms), color='#ec6066ff', linewidth=8, label='LogsAlarms')
axes[0].plot(LogsEvents['DateTime'], [1] * len(LogsEvents), color='#6699ccff', linewidth=8, label='LogsEvents')

axes[0].grid(axis='both', linestyle='--', linewidth=0.5, alpha=0.7)  # Add auxiliary x-axis grid

#Get min and max date for each dataframe
mindateS = LogsStandard['DateTime'].min()
maxdateS = LogsStandard['DateTime'].max()

mindateA = LogsAlarms['DateTime'].min()
maxdateA = LogsAlarms['DateTime'].max()

mindateE = LogsEvents['DateTime'].min()
maxdateE = LogsEvents['DateTime'].max()

# Filter datetime values to get unique dates ignoring time
dates = [mindateS, mindateA, mindateE, maxdateS, maxdateA, maxdateE]
ticks = list(set(dt.date() for dt in dates))
ticks.sort()

# Create a new list to store filtered dates
filtered_dates = [ticks[0]]
# Iterate through the sorted date list and delete dates near less than x days
for i in range(1, len(ticks)):
    if (ticks[i] - filtered_dates[-1]).days >= 2:
        filtered_dates.append(ticks[i])
# Format the unique dates as "01/01/23" strings
xtick_labels = [date.strftime('%m/%d/%y') for date in filtered_dates]

axes[0].set_xticks(filtered_dates)
axes[0].set_xticklabels(xtick_labels, rotation=45)
axes[0].set_yticks([1, 2, 3])
axes[0].set_yticklabels(['Events Logs', 'Alarms Logs', 'Standard Logs'])

# ----------------------------------------------------------------------------- Row 2: Multiple y-axes
ax1 = axes[1]
ax1.set_title('Main variables trend')
ax1.grid(axis='both', linestyle='--', linewidth=0.5, alpha=0.7)  # Add auxiliary x-axis grid

# Data
x = LogsStandard['DateTime']
y1 = LogsStandard['FT_VolumeFlow']
y2 = LogsStandard['TT2']
y3 = LogsStandard['VT']

#y1
ax1.plot(x, y1, color='#6699ccff', label='FT_VolumeFlow')
ax1.set_ylabel('FT_VolumeFlow', color='#6699ccff')
ax1.tick_params(axis='y', labelcolor='#6699ccff')

#y2
ax2 = ax1.twinx()
ax2.plot(x, y2, color='#ec6066ff', label='TT2')
ax2.set_ylabel('TT2', color='#ec6066ff')
ax2.tick_params(axis='y', labelcolor='#ec6066ff')

#y3
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 50))
ax3.plot(x, y3, color='#FFCC00', label='VT')
ax3.set_ylabel('Viscosity', color='#FFCC00')
ax3.tick_params(axis='y', labelcolor='#FFCC00')

# Set fewer ticks on the x-axis
min_date = LogsStandard['DateTime'].min()
max_date = LogsStandard['DateTime'].max()
num_ticks = 10  # Choose the number of ticks you prefer
x_ticks = pd.date_range(start=min_date, end=max_date, freq='D')
x_ticks = x_ticks[::len(x_ticks) // num_ticks]  # Select every Nth tick

# Set custom tick positions and labels on the x-axis
ax1.set_xticks(x_ticks)
ax1.set_xticklabels(x_ticks.strftime('%m/%d/%y'), rotation=45)  # Format and rotate tick labels

# Combine legends for the second row
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(hspace=0.5)  # Adjust vertical spacing between rows

plt.show()

In [ ]:
import customtkinter as ctk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tkcalendar
import matplotlib.pyplot as plt

# Define your custom style
custom_dark_style = {
    'axes.facecolor': '#24242424',  # Custom background color / DEFAULT '#1e1e1e'
    'axes.edgecolor': '#ffffff',  # Color of axes edges / DEFAULT '#ffffff'
    'axes.labelcolor': '#ffffff',  # Color of axes labels / DEFAULT 
    'text.color': '#ffffff',  # Color of text / DEFAULT '#ffffff'
    'xtick.color': '#ffffff',  # Color of x-axis ticks / DEFAULT '#ffffff'
    'ytick.color': '#ffffff',  # Color of y-axis ticks / DEFAULT '#ffffff'
    'figure.facecolor': '#1e1e1e',  # Figure background color / DEFAULT '#1e1e1e'
    'figure.edgecolor': '#1e1e1e',  # Figure edge color / DEFAULT '#1e1e1e'
    'axes.grid': True,  # Show grid / DEFAULT True
    'grid.color': '#333333',  # Color of grid lines / DEFAULT '#333333'
}

def create_two_row_plot():

    plt.style.use(custom_dark_style)
    #plt.style.use('default')
    #plt.style.use('dark_background')

    # Create a two-row plot
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 6), gridspec_kw={'height_ratios': [1, 5]})

    # ---------------------------------------------------------------------------- Row 1: Line plots
    axes[0].set_title('Date range of Log files')
    axes[0].plot(LogsStandard['DateTime'], [3] * len(LogsStandard), color='#FFCC00', linewidth=8, label='LogsStandard')

    axes[0].plot(LogsAlarms['DateTime'], [2] * len(LogsAlarms), color='#ec6066ff', linewidth=8, label='LogsAlarms')
    axes[0].plot(LogsEvents['DateTime'], [1] * len(LogsEvents), color='#6699ccff', linewidth=8, label='LogsEvents')

    axes[0].grid(axis='both', linestyle='--', linewidth=0.5, alpha=0.7)  # Add auxiliary x-axis grid

    #Get min and max date for each dataframe
    mindateS = LogsStandard['DateTime'].min()
    maxdateS = LogsStandard['DateTime'].max()

    mindateA = LogsAlarms['DateTime'].min()
    maxdateA = LogsAlarms['DateTime'].max()

    mindateE = LogsEvents['DateTime'].min()
    maxdateE = LogsEvents['DateTime'].max()

    # Filter datetime values to get unique dates ignoring time
    dates = [mindateS, mindateA, mindateE, maxdateS, maxdateA, maxdateE]
    ticks = list(set(dt.date() for dt in dates))
    ticks.sort()

    # Create a new list to store filtered dates
    filtered_dates = [ticks[0]]
    # Iterate through the sorted date list and delete dates near less than x days
    for i in range(1, len(ticks)):
        if (ticks[i] - filtered_dates[-1]).days >= 2:
            filtered_dates.append(ticks[i])
    # Format the unique dates as "01/01/23" strings
    xtick_labels = [date.strftime('%m/%d/%y') for date in filtered_dates]

    axes[0].set_xticks(filtered_dates)
    axes[0].set_xticklabels(xtick_labels, rotation=45)
    axes[0].set_yticks([1, 2, 3])
    axes[0].set_yticklabels(['Events Logs', 'Alarms Logs', 'Standard Logs'])

    # ----------------------------------------------------------------------------- Row 2: Multiple y-axes
    ax1 = axes[1]
    ax1.set_title('Main variables trend')
    ax1.grid(axis='both', linestyle='--', linewidth=0.5, alpha=0.7)  # Add auxiliary x-axis grid

    # Data
    x = LogsStandard['DateTime']
    y1 = LogsStandard['FT_VolumeFlow']
    y2 = LogsStandard['TT2']
    y3 = LogsStandard['VT']

    #y1
    ax1.plot(x, y1, color='#6699ccff', label='FT_VolumeFlow')
    ax1.set_ylabel('FT_VolumeFlow', color='#6699ccff')
    ax1.tick_params(axis='y', labelcolor='#6699ccff')

    #y2
    ax2 = ax1.twinx()
    ax2.plot(x, y2, color='#ec6066ff', label='TT2')
    ax2.set_ylabel('TT2', color='#ec6066ff')
    ax2.tick_params(axis='y', labelcolor='#ec6066ff')

    #y3
    ax3 = ax1.twinx()
    ax3.spines['right'].set_position(('outward', 50))
    ax3.plot(x, y3, color='#FFCC00', label='VT')
    ax3.set_ylabel('Viscosity', color='#FFCC00')
    ax3.tick_params(axis='y', labelcolor='#FFCC00')

    # Set fewer ticks on the x-axis
    min_date = LogsStandard['DateTime'].min()
    max_date = LogsStandard['DateTime'].max()
    num_ticks = 10  # Choose the number of ticks you prefer
    x_ticks = pd.date_range(start=min_date, end=max_date, freq='D')
    x_ticks = x_ticks[::len(x_ticks) // num_ticks]  # Select every Nth tick

    # Set custom tick positions and labels on the x-axis
    ax1.set_xticks(x_ticks)
    ax1.set_xticklabels(x_ticks.strftime('%m/%d/%y'), rotation=45)  # Format and rotate tick labels

    # Adjust layout
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.5)  # Adjust vertical spacing between rows

    return fig

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Custom Tkinter App")

        self.aux_plot = ctk.CTkButton(self.root, text="Show Auxiliary Plot", command=self.show_plot)
        self.aux_plot.grid(row=0, column=0, columnspan=2, padx=20, pady=20)

        self.cal1_text = ctk.CTkLabel(self.root, text='From:')
        self.cal1_text.grid(row=1, column=0, padx=20, pady=2, sticky="nw") 
        self.cal1d = tkcalendar.Calendar(self.root)
        self.cal1d.grid(row=2, column=0, padx=20, pady=2)
        self.cal1t = ctk.CTkOptionMenu(self.root)
        self.cal1t.grid(row=3, column=0, padx=20, pady=2)

        self.cal1_text = ctk.CTkLabel(self.root, text='To:')
        self.cal1_text.grid(row=1, column=1, padx=20, pady=2, sticky="nw")
        self.cal2d = tkcalendar.Calendar(self.root)
        self.cal2d.grid(row=2, column=1, padx=20, pady=2)
        self.cal2t = ctk.CTkOptionMenu(self.root)
        self.cal2t.grid(row=3, column=1, padx=20, pady=2)

        self.action_t3 = ctk.CTkButton(self.root, text="Generate and save Plot", command=self.show_plot)
        self.action_t3.grid(row=4, column=1, padx=20, pady=20, sticky="se")

    def show_plot(self):
        plot_fig = create_two_row_plot()
        plot_window = ctk.CTkToplevel(self.root)
        plot_window.resizable(width=False, height=False)
        plot_window.title("Auxiliary Plot")
        
        canvas = FigureCanvasTkAgg(plot_fig, master=plot_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

if __name__ == "__main__":
    root = ctk.CTk()
    app = App(root)
    root.mainloop()


# Logger

In [1]:
import logging
import os

PATH = os.getcwd()
file_path = os.path.join(PATH, 'test1.log')

#Create and configure logger
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
logging.basicConfig(filename= file_path,
                    level= logging.DEBUG,
                    format = LOG_FORMAT)

logger = logging.getLogger()

#test messages
logger.debug("This is a message. ")
logger.info("This is useful info. ")
logger.warning("I can't do that. ")
logger.error("I could not find the file")
logger.critical("The internet is down!")
